In [24]:
import numpy as np
import keras
import os
from pathlib import Path

In [25]:
batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.
# Path to the data txt file on disk.
data_path = "ara.txt"

In [26]:
# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path, "r", encoding="utf-8") as f:
    lines = f.read().split("\n")
for line in lines[: min(num_samples, len(lines))]:
    input_text, target_text, _ = line.split("\t")
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = "\t" + target_text + "\n"
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print("Number of samples:", len(input_texts))
print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_decoder_tokens)
print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for outputs:", max_decoder_seq_length)

input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype="float32",
)
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype="float32",
)
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype="float32",
)

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.0
    encoder_input_data[i, t + 1 :, input_token_index[" "]] = 1.0
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.0
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
    decoder_input_data[i, t + 1 :, target_token_index[" "]] = 1.0
    decoder_target_data[i, t:, target_token_index[" "]] = 1.0

Number of samples: 10000
Number of unique input tokens: 71
Number of unique output tokens: 94
Max sequence length for inputs: 36
Max sequence length for outputs: 54


In [27]:
# Define an input sequence and process it.
encoder_inputs = keras.Input(shape=(None, num_encoder_tokens))
encoder = keras.layers.LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = keras.Input(shape=(None, num_decoder_tokens))

# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = keras.layers.LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = keras.layers.Dense(num_decoder_tokens, activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [ ]:
model.compile(
    optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"]
)
model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=batch_size,
    epochs=1000,
    validation_split=0.2,
)
# Save model
model.save("s2s_model.keras")

Epoch 1/1000
125/125 [==============================] - 8s 30ms/step - loss: 1.3843 - accuracy: 0.7023 - val_loss: 1.6575 - val_accuracy: 0.5838
Epoch 2/1000
125/125 [==============================] - 2s 13ms/step - loss: 1.1193 - accuracy: 0.7157 - val_loss: 1.6379 - val_accuracy: 0.5933
Epoch 3/1000
125/125 [==============================] - 2s 13ms/step - loss: 1.0377 - accuracy: 0.7302 - val_loss: 1.4552 - val_accuracy: 0.6150
Epoch 4/1000
125/125 [==============================] - 3s 20ms/step - loss: 0.9632 - accuracy: 0.7474 - val_loss: 1.3649 - val_accuracy: 0.6396
Epoch 5/1000
125/125 [==============================] - 2s 13ms/step - loss: 0.9425 - accuracy: 0.7537 - val_loss: 1.3358 - val_accuracy: 0.6475
Epoch 6/1000
125/125 [==============================] - 2s 19ms/step - loss: 0.8926 - accuracy: 0.7635 - val_loss: 1.3062 - val_accuracy: 0.6523
Epoch 7/1000
125/125 [==============================] - 2s 16ms/step - loss: 0.8692 - accuracy: 0.7682 - val_loss: 1.2657 - val_ac

In [28]:
# Define sampling models
# Restore the model and construct the encoder and decoder.
model = keras.models.load_model("s2s_model.keras")

encoder_inputs = model.input[0]  # input_1
encoder_outputs, state_h_enc, state_c_enc = model.layers[2].output  # lstm_1
encoder_states = [state_h_enc, state_c_enc]
encoder_model = keras.Model(encoder_inputs, encoder_states)

decoder_inputs = model.input[1]  # input_2
decoder_state_input_h = keras.Input(shape=(latent_dim,))
decoder_state_input_c = keras.Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_lstm = model.layers[3]
decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs
)
decoder_states = [state_h_dec, state_c_dec]
decoder_dense = model.layers[4]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = keras.Model(
    [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states
)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())


def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq, verbose=0)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index["\t"]] = 1.0

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ""
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value, verbose=0
        )

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.0

        # Update states
        states_value = [h, c]
    return decoded_sentence

In [29]:
for seq_index in range(20):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index : seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print("-")
    print("Input sentence:", input_texts[seq_index])
    print("Decoded sentence:", decoded_sentence)

-
Input sentence: Hi.
Decoded sentence: مرحبًا.

-
Input sentence: Run!
Decoded sentence: اركض!

-
Input sentence: Duck!
Decoded sentence: اخفضي رأسك!

-
Input sentence: Duck!
Decoded sentence: اخفضي رأسك!

-
Input sentence: Duck!
Decoded sentence: اخفضي رأسك!

-
Input sentence: Help!
Decoded sentence: النجدة!

-
Input sentence: Jump!
Decoded sentence: اقفز!

-
Input sentence: Stop!
Decoded sentence: قف!

-
Input sentence: Stop!
Decoded sentence: قف!

-
Input sentence: Wait!
Decoded sentence: إنتظر

-
Input sentence: Go on.
Decoded sentence: استمر.

-
Input sentence: Go on.
Decoded sentence: استمر.

-
Input sentence: Hello!
Decoded sentence: مرحباً.

-
Input sentence: Hello.
Decoded sentence: مرحباً.

-
Input sentence: Hello.
Decoded sentence: مرحباً.

-
Input sentence: Hurry!
Decoded sentence: استعجل!

-
Input sentence: Hurry!
Decoded sentence: استعجل!

-
Input sentence: I see.
Decoded sentence: انا اري

-
Input sentence: I won!
Decoded sentence: أنا فُزت!

-
Input sentence: Relax.
De

In [37]:
while True:
    try:
        user_input = input("Enter a word or sentence: ")  # Prompt user for input

        # Get the sequence index of the input text
        seq_index = -1
        for i, text in enumerate(input_texts):
            if user_input.lower() in text.lower():
                seq_index = i
                break

        if seq_index == -1:
            print("The word or sentence is not found in the training set.")
            continue  # Restart loop to prompt for valid input

        # Take one sequence (part of the training set) for decoding
        input_seq = encoder_input_data[seq_index: seq_index + 1]
        decoded_sentence = decode_sequence(input_seq)

        print("-" * 30)
        print("Input sentence:", input_texts[seq_index])
        print("Decoded sentence:", decoded_sentence)

        # Prompt user to continue or exit
        choice = input("Do you want to decode another sequence? (y/n): ").lower()
        if choice != 'y':
            print("Exiting. Goodbye!")
            break  # Exit loop if user chooses not to continue

    except Exception as e:
        print("An error occurred:", str(e))
        print("Please enter a valid word or sentence.")


Enter a word or sentence: go on
------------------------------
Input sentence: Go on.
Decoded sentence: استمر.

Do you want to decode another sequence? (y/n): y
Enter a word or sentence: run
------------------------------
Input sentence: Run!
Decoded sentence: اركض!

Do you want to decode another sequence? (y/n): n
Exiting. Goodbye!
